In [1]:
import os
import json
import requests
import zipfile
import kaggle
import pandas as pd
from datetime import datetime
from IPython.display import display

In [ ]:
#kaggle_api = 'd216ca4ba0e7544cb2adafaf9c4d3ab0'

# Get Kaggle Data
##### https://www.kaggle.com/datasets/tobycrabtree/nfl-scores-and-betting-data?resource=download&select=spreadspoke_scores.csv

- all games since 1966
- teams name/abbreviation/conference history
- stadiums

In [2]:
### Download raw data

# Authenticate with Kaggle using the API token in ~/.kaggle/kaggle.json
kaggle.api.authenticate()

# Download the dataset
kaggle.api.dataset_download_files('tobycrabtree/nfl-scores-and-betting-data', path='./data-raw-kaggle-1', unzip=True)

print("Dataset downloaded successfully.")

Dataset downloaded successfully.


In [ ]:
# ### Merge everything

# # Load the original dataset
# df = pd.read_csv('./data-raw-kaggle/spreadspoke_scores.csv')

# # Load the additional datasets
# stadiums_df = pd.read_csv('./data-raw-kaggle/nfl_stadiums.csv', encoding='ISO-8859-1')
# teams_df = pd.read_csv('./data-raw-kaggle/nfl_teams.csv')

# # Merge the stadium information into the original dataset
# merged_df = pd.merge(df, stadiums_df, how='left', left_on='stadium', right_on='stadium_name')

# # Merge the team information for 'team_home' and 'team_away'
# merged_df = pd.merge(merged_df, teams_df, how='left', left_on='team_home', right_on='team_name', suffixes=('', '_home'))
# merged_df = pd.merge(merged_df, teams_df, how='left', left_on='team_away', right_on='team_name', suffixes=('', '_away'))

# # Save the merged DataFrame to a new CSV file
# merged_df.to_csv('./data-raw-kaggle/merged_kaggle.csv', index=False)


In [3]:
### Merge just stuff I need

# Load the original dataset
df = pd.read_csv('./data-raw-kaggle/spreadspoke_scores.csv')

# Load the additional dataset for teams
teams_df = pd.read_csv('./data-raw-kaggle/nfl_teams.csv')

# Columns to merge from nfl_teams.csv
cols_to_merge = ['team_id', 'team_id_pfr', 'team_division', 'team_division_pre2002']

# Add 'team_name' for merging
cols_to_merge_with_name = cols_to_merge + ['team_name']

# Merge the team information for 'team_home' and 'team_away'
merged_df = pd.merge(df, teams_df[cols_to_merge_with_name], how='left', left_on='team_home', right_on='team_name')
merged_df.rename(columns={col: 'home_' + col for col in cols_to_merge}, inplace=True)

merged_df = pd.merge(merged_df, teams_df[cols_to_merge_with_name], how='left', left_on='team_away', right_on='team_name')
merged_df.rename(columns={col: 'away_' + col for col in cols_to_merge}, inplace=True)

# Drop the extra 'team_name' columns
merged_df.drop(['team_name_x', 'team_name_y'], axis=1, inplace=True)

# Save the merged DataFrame to a new CSV file
merged_df.to_csv('./data-raw-kaggle/merged_kaggle.csv', index=False)


In [4]:
### Check missing odds data

# Load the CSV into a DataFrame
merged_df = pd.read_csv('./data-raw-kaggle/merged_kaggle.csv')

# Convert the 'schedule_date' column to datetime format
merged_df['schedule_date'] = pd.to_datetime(merged_df['schedule_date'])

# Filter the data for games in the last 10 years
start_date = datetime(2013, 9, 25)  # 10 years from today's date (Sep 25, 2023)
end_date = datetime(2023, 9, 25)  # Today's date (Sep 25, 2023)
last_10_years_df = merged_df[(merged_df['schedule_date'] >= start_date) & (merged_df['schedule_date'] <= end_date)]

# Filter the rows where any of the columns 'team_favorite_id', 'spread_favorite', or 'over_under_line' have missing values
missing_data_rows = last_10_years_df[last_10_years_df[['team_favorite_id', 'spread_favorite', 'over_under_line']].isnull().any(axis=1)]

# Print the rows with missing data
#print(missing_data_rows.head(15))
display(missing_data_rows.head(15))


schedule_date  schedule_season schedule_week  schedule_playoff  \
13549    2023-09-24             2023             3             False   
13550    2023-09-24             2023             3             False   
13551    2023-09-24             2023             3             False   
13552    2023-09-24             2023             3             False   
13553    2023-09-24             2023             3             False   
13554    2023-09-24             2023             3             False   
13555    2023-09-24             2023             3             False   
13556    2023-09-24             2023             3             False   
13557    2023-09-24             2023             3             False   
13558    2023-09-24             2023             3             False   
13559    2023-09-24             2023             3             False   
13560    2023-09-24             2023             3             False   
13561    2023-09-24             2023             3             False   
13562    2023-09-25             2023             3             False   
13563    2023-09-25             2023             3             False   

                   team_home  score_home  score_away             team_away  \
13549      Arizona Cardinals         NaN         NaN        Dallas Cowboys   
13550       Baltimore Ravens         NaN         NaN    Indianapolis Colts   
13551       Cleveland Browns         NaN         NaN      Tennessee Titans   
13552          Detroit Lions         NaN         NaN       Atlanta Falcons   
13553      Green Bay Packers         NaN         NaN    New Orleans Saints   
13554   Jacksonville Jaguars         NaN         NaN        Houston Texans   
13555     Kansas City Chiefs         NaN         NaN         Chicago Bears   
13556      Las Vegas Raiders         NaN         NaN   Pittsburgh Steelers   
13557         Miami Dolphins         NaN         NaN        Denver Broncos   
13558      Minnesota Vikings         NaN         NaN  Los Angeles Chargers   
13559          New York Jets         NaN         NaN  New England Patriots   
13560       Seattle Seahawks         NaN         NaN     Carolina Panthers   
13561  Washington Commanders         NaN         NaN         Buffalo Bills   
13562     Cincinnati Bengals         NaN         NaN      Los Angeles Rams   
13563   Tampa Bay Buccaneers         NaN         NaN   Philadelphia Eagles   

      team_favorite_id  spread_favorite  ... weather_humidity weather_detail  \
13549              NaN              NaN  ...              NaN         indoor   
13550              NaN              NaN  ...              NaN            NaN   
13551              NaN              NaN  ...              NaN            NaN   
13552              NaN              NaN  ...              NaN         indoor   
13553              NaN              NaN  ...              NaN            NaN   
13554              NaN              NaN  ...              NaN            NaN   
13555              NaN              NaN  ...              NaN            NaN   
13556              NaN              NaN  ...              NaN         indoor   
13557              NaN              NaN  ...              NaN            NaN   
13558              NaN              NaN  ...              NaN         indoor   
13559              NaN              NaN  ...              NaN            NaN   
13560              NaN              NaN  ...              NaN            NaN   
13561              NaN              NaN  ...              NaN            NaN   
13562              NaN              NaN  ...              NaN            NaN   
13563              NaN              NaN  ...              NaN            NaN   

       home_team_id  home_team_id_pfr  home_team_division  \
13549           ARI               CRD            NFC West   
13550           BAL               RAV           AFC North   
13551           CLE               CLE           AFC North   
13552           DET               DET           NFC North   
13553            G

# Analysis

### Check Super Bowl Winner

In [5]:
# Load the CSV file into a DataFrame
merged_df = pd.read_csv('./data-raw-kaggle/merged_kaggle.csv')

# The year you are interested in
year_of_interest = 2021  # Change this value as needed

# Convert the 'schedule_date' column to datetime format
merged_df['schedule_date'] = pd.to_datetime(merged_df['schedule_date'])

# Filter the data for the specified season
season_df = merged_df[merged_df['schedule_season'] == year_of_interest].copy()

# Create a new column 'winner' to indicate which team won: 'Home', 'Away', or 'Tie'
season_df['winner'] = 'Tie'
season_df.loc[season_df['score_home'] > season_df['score_away'], 'winner'] = 'Home'
season_df.loc[season_df['score_away'] > season_df['score_home'], 'winner'] = 'Away'

# Filter for the playoff games and find the game with the latest date (Super Bowl)
playoffs_df = season_df[season_df['schedule_playoff'] == True]
super_bowl = playoffs_df[playoffs_df['schedule_date'] == playoffs_df['schedule_date'].max()]

# Determine the winner of the Super Bowl for the specified season
if super_bowl.empty:
    print("No Super Bowl data available for the {} season".format(year_of_interest))
else:
    if super_bowl.iloc[0]['winner'] == 'Home':
        print("Super Bowl Winner for {} season:".format(year_of_interest), super_bowl.iloc[0]['team_home'])
    elif super_bowl.iloc[0]['winner'] == 'Away':
        print("Super Bowl Winner for {} season:".format(year_of_interest), super_bowl.iloc[0]['team_away'])
    else:
        print("The Super Bowl for the {} season ended in a tie".format(year_of_interest))
        
# Display more details about the Super Bowl game in a table
display(super_bowl[['schedule_date', 'team_home', 'team_away', 'score_home', 'score_away']])


Super Bowl Winner for 2021 season: Los Angeles Rams


schedule_date           team_home         team_away  score_home  \
13231    2022-02-13  Cincinnati Bengals  Los Angeles Rams        20.0   

       score_away  
13231        23.0

### Cowboys Games in 2022 Season

In [6]:
# Load the CSV file into a DataFrame
merged_df = pd.read_csv('./data-raw-kaggle/merged_kaggle.csv')

# Convert the 'schedule_date' column to datetime format
merged_df['schedule_date'] = pd.to_datetime(merged_df['schedule_date'])

# Filter the data for the 2022 season
season_2022_df = merged_df[merged_df['schedule_season'] == 2022].copy()

# Create a new column 'winner' to indicate which team won: 'Home', 'Away', or 'Tie'
season_2022_df['winner'] = 'Tie'
season_2022_df.loc[season_2022_df['score_home'] > season_2022_df['score_away'], 'winner'] = 'Home'
season_2022_df.loc[season_2022_df['score_away'] > season_2022_df['score_home'], 'winner'] = 'Away'

# Filter the data for Dallas Cowboys games in the 2022 season
cowboys_games_2022 = season_2022_df[(season_2022_df['team_home'] == 'Dallas Cowboys') | 
                                    (season_2022_df['team_away'] == 'Dallas Cowboys')].copy()

# Add a column to indicate if the Cowboys won, lost, or if the game was a tie
def cowboys_result(row):
    if row['team_home'] == 'Dallas Cowboys':
        if row['winner'] == 'Home':
            return 'Win'
        elif row['winner'] == 'Away':
            return 'Loss'
        else:
            return 'Tie'
    elif row['team_away'] == 'Dallas Cowboys':
        if row['winner'] == 'Home':
            return 'Loss'
        elif row['winner'] == 'Away':
            return 'Win'
        else:
            return 'Tie'

cowboys_games_2022['Cowboys_Result'] = cowboys_games_2022.apply(cowboys_result, axis=1)

# Select only the relevant columns for display
cowboys_games_2022_relevant = cowboys_games_2022[['schedule_date', 'team_home', 'team_away', 'score_home', 'score_away', 'Cowboys_Result']]
sorted_cowboys_games = cowboys_games_2022_relevant.sort_values(by='schedule_date')

# Display the DataFrame in a neat table format
display(sorted_cowboys_games)



schedule_date              team_home              team_away  score_home  \
13238    2022-09-11         Dallas Cowboys   Tampa Bay Buccaneers         3.0   
13251    2022-09-18         Dallas Cowboys     Cincinnati Bengals        20.0   
13279    2022-09-26        New York Giants         Dallas Cowboys        16.0   
13284    2022-10-02         Dallas Cowboys  Washington Commanders        25.0   
13304    2022-10-09       Los Angeles Rams         Dallas Cowboys        10.0   
13322    2022-10-16    Philadelphia Eagles         Dallas Cowboys        26.0   
13330    2022-10-23         Dallas Cowboys          Detroit Lions        24.0   
13343    2022-10-30         Dallas Cowboys          Chicago Bears        49.0   
13371    2022-11-13      Green Bay Packers         Dallas Cowboys        31.0   
13390    2022-11-20      Minnesota Vikings         Dallas Cowboys         3.0   
13396    2022-11-24         Dallas Cowboys        New York Giants        28.0   
13417    2022-12-04         Dallas Cowboys     Indianapolis Colts        54.0   
13430    2022-12-11         Dallas Cowboys         Houston Texans        27.0   
13448    2022-12-18   Jacksonville Jaguars         Dallas Cowboys        40.0   
13461    2022-12-24         Dallas Cowboys    Philadelphia Eagles        40.0   
13472    2022-12-29       Tennessee Titans         Dallas Cowboys        13.0   
13502    2023-01-08  Washington Commanders         Dallas Cowboys        26.0   
13508    2023-01-16   Tampa Bay Buccaneers         Dallas Cowboys        14.0   
13512    2023-01-22    San Francisco 49ers         Dallas Cowboys        19.0   

       score_away Cowboys_Result  
13238        19.0           Loss  
13251        17.0            Win  
13279        23.0            Win  
13284        10.0            Win  
13304        22.0            Win  
13322        17.0           Loss  
13330         6.0            Win  
13343        29.0            Win  
13371        28.0           Loss  
13390        40.0            Win  
13396        20.0            Win  
13417        19.0            Win  
13430        23.0            Win  
13448        34.0           Loss  
13461        34.0            Win  
13472        27.0            Win  
13502         6.0           Loss  
13508        31.0            Win  
13512        12.0           Loss

### Trends in Pick'em Games at Home/Away

In [ ]:
# Load the CSV file into a DataFrame
merged_df = pd.read_csv('./data-raw-kaggle/merged_kaggle.csv')

# Convert the 'schedule_date' column to datetime format
merged_df['schedule_date'] = pd.to_datetime(merged_df['schedule_date'])

# Filter the data for games in the last 10 years and create a copy
start_date = datetime(2013, 9, 25)
end_date = datetime(2023, 9, 25)
last_10_years_df = merged_df[(merged_df['schedule_date'] >= start_date) & (merged_df['schedule_date'] <= end_date)].copy()

# Create a new column 'winner' to indicate which team won: 'Home', 'Away', or 'Tie'
last_10_years_df.loc[:, 'winner'] = 'Tie'
last_10_years_df.loc[last_10_years_df['score_home'] > last_10_years_df['score_away'], 'winner'] = 'Home'
last_10_years_df.loc[last_10_years_df['score_away'] > last_10_years_df['score_home'], 'winner'] = 'Away'

# Count the occurrences of each type of winner: 'Home', 'Away', or 'Tie'
winner_counts = last_10_years_df['winner'].value_counts(normalize=True) * 100

# Filter rows for pick'em and close games
pickem_and_close_games = last_10_years_df[(last_10_years_df['team_favorite_id'] == 'PICK') |
                                          (last_10_years_df['spread_favorite'].abs() <= 1)]

# Count the occurrences of each type of winner in pick'em and close games
winner_counts_pickem_close = pickem_and_close_games['winner'].value_counts(normalize=True) * 100

print("General Trends in the Last 10 Years:", winner_counts)
print("Trends in Pick'em and Close Games:", winner_counts_pickem_close)
